In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load your data
df = pd.read_csv('2August.csv')


# Define features and target
X = df.drop('Disease', axis=1)
y = df['Disease']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert back to DataFrame to match the format
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)


In [2]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [3]:
my_feature_columns = []
for key in X_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [4]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[128, 64, 32],  # Three hidden layers with 128, 64, and 32 nodes respectively
    n_classes=9,  # Assuming 9 classes including 'Healthy'
    optimizer=lambda: tf.keras.optimizers.Adam(learning_rate=0.001)  # Use Adam optimizer with a custom learning rate
)

classifier.train(
    input_fn=lambda: input_fn(X_train, y_train, training=True),
    steps=10000)  # Increase the number of steps

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\bioha\\AppData\\Local\\Temp\\tmpwtw232le', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_v

In [5]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(X_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-08-02T16:11:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.29870s
INFO:tensorflow:Finished evaluation at 2024-08-02-16:11:32
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9647577, average_loss = 0.32144675, global_step = 10000, loss = 0.33598578
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000

Test set accuracy: 0.965



In [6]:
def input_fn(features, batch_size=256):
    """An input function for prediction"""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Define the features you need to collect from the user
features = ['Gender', 'Age', 'GLUCOSE', 'HBA1C', 'CREATININE', 'BUN', 'HGB', 'Ferr',
            'TROPONIN_I', 'CK', 'CK_MB', 'TRIGLYCERIDE', 'HDL_CHOLESTEROL',
            'LDL_CHOLESTEROL', 'CRP', 'PCT', 'FT4', 'TSH']

# Collect user input for each feature
predict_features = {}
print("Please type numeric values as prompted.")
for feature in features:
    while True:
        try:
            val = float(input(f"{feature}: "))
            predict_features[feature] = [val]
            break
        except ValueError:
            print(f"Invalid input for {feature}. Please enter a numeric value.")

# Make predictions
predictions = classifier.predict(input_fn=lambda: input_fn(predict_features))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        class_id, 100 * probability))

Please type numeric values as prompted.


Gender:  0
Age:  65
GLUCOSE:  136
HBA1C:  6.9
CREATININE:  0.5
BUN:  12
HGB:  13.4
Ferr:  116.4
TROPONIN_I:  3.1
CK:  62.9
CK_MB:  5.6
TRIGLYCERIDE:  98.0
HDL_CHOLESTEROL:  23.4
LDL_CHOLESTEROL:  103.5
CRP:  4.5
PCT:  0.34
FT4:  0.81
TSH:  1.201


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "2" (100.0%)


In [7]:
def input_fn(features, batch_size=256):
    """An input function for prediction"""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Define the features you need to collect from the user
features = ['Gender', 'Age', 'GLUCOSE', 'HBA1C', 'CREATININE', 'BUN', 'HGB', 'Ferr',
            'TROPONIN_I', 'CK', 'CK_MB', 'TRIGLYCERIDE', 'HDL_CHOLESTEROL',
            'LDL_CHOLESTEROL', 'CRP', 'PCT', 'FT4', 'TSH']

# Collect user input for each feature
predict_features = {}
print("Please type numeric values as prompted.")
for feature in features:
    while True:
        try:
            val = float(input(f"{feature}: "))
            predict_features[feature] = [val]
            break
        except ValueError:
            print(f"Invalid input for {feature}. Please enter a numeric value.")

# Make predictions
predictions = classifier.predict(input_fn=lambda: input_fn(predict_features))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        class_id, 100 * probability))

Please type numeric values as prompted.


Gender:  1
Age:  45
GLUCOSE:  110
HBA1C:  6
CREATININE:  0.8
BUN:  13.1
HGB:  13.2
Ferr:  150.1
TROPONIN_I:  11.1
CK:  84.5
CK_MB:  18.4
TRIGLYCERIDE:  103.5
HDL_CHOLESTEROL:  


Invalid input for HDL_CHOLESTEROL. Please enter a numeric value.


HDL_CHOLESTEROL:  43.1
LDL_CHOLESTEROL:  110.8
CRP:  43.1
PCT:  0.34
FT4:  0.58
TSH:  1.12


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "2" (100.0%)


In [8]:
def input_fn(features, batch_size=256):
    """An input function for prediction"""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Define the features you need to collect from the user
features = ['Gender', 'Age', 'GLUCOSE', 'HBA1C', 'CREATININE', 'BUN', 'HGB', 'Ferr',
            'TROPONIN_I', 'CK', 'CK_MB', 'TRIGLYCERIDE', 'HDL_CHOLESTEROL',
            'LDL_CHOLESTEROL', 'CRP', 'PCT', 'FT4', 'TSH']

# Collect user input for each feature
predict_features = {}
print("Please type numeric values as prompted.")
for feature in features:
    while True:
        try:
            val = float(input(f"{feature}: "))
            predict_features[feature] = [val]
            break
        except ValueError:
            print(f"Invalid input for {feature}. Please enter a numeric value.")

# Make predictions
predictions = classifier.predict(input_fn=lambda: input_fn(predict_features))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        class_id, 100 * probability))

Please type numeric values as prompted.


Gender:  1
Age:  45
GLUCOSE:  110
HBA1C:  4.5
CREATININE:  0.8
BUN:  13.1
HGB:  13.2
Ferr:  150.1
TROPONIN_I:  11.1
CK:  84.5
CK_MB:  18.4
TRIGLYCERIDE:  103.5
HDL_CHOLESTEROL:  43.1
LDL_CHOLESTEROL:  110.8
CRP:  43.1
PCT:  0.34
FT4:  0.58
TSH:  1.12


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "2" (100.0%)


In [9]:
def input_fn(features, batch_size=256):
    """An input function for prediction"""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Define the features you need to collect from the user
features = ['Gender', 'Age', 'GLUCOSE', 'HBA1C', 'CREATININE', 'BUN', 'HGB', 'Ferr',
            'TROPONIN_I', 'CK', 'CK_MB', 'TRIGLYCERIDE', 'HDL_CHOLESTEROL',
            'LDL_CHOLESTEROL', 'CRP', 'PCT', 'FT4', 'TSH']

# Collect user input for each feature
predict_features = {}
print("Please type numeric values as prompted.")
for feature in features:
    while True:
        try:
            val = float(input(f"{feature}: "))
            predict_features[feature] = [val]
            break
        except ValueError:
            print(f"Invalid input for {feature}. Please enter a numeric value.")

# Make predictions
predictions = classifier.predict(input_fn=lambda: input_fn(predict_features))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        class_id, 100 * probability))

Please type numeric values as prompted.


Gender:  1
Age:  45
GLUCOSE:  95
HBA1C:  4.5
CREATININE:  0.8
BUN:  13.1
HGB:  13.2
Ferr:  150.1
TROPONIN_I:  11.1
CK:  84.5
CK_MB:  18.4
TRIGLYCERIDE:  103.5
HDL_CHOLESTEROL:  43.1
LDL_CHOLESTEROL:  110.8
CRP:  43.1
PCT:  0.34
FT4:  0.58
TSH:  1.12


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\bioha\AppData\Local\Temp\tmpwtw232le\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "2" (100.0%)
